# Nova Hybrid Reasoning with LangChain

This notebook demonstrates how to use Amazon Nova 2.0's hybrid reasoning capabilities with LangChain's ChatBedrockConverse.

Nova 2.0 models support reasoning configuration that allows the model to think through problems step-by-step. The reasoning process is controlled by the `maxReasoningEffort` parameter with three options: "low", "medium", and "high".

The reasoning budget sets a limit on how many tokens should be used in the reasoning process, controlling how deeply the model thinks through problems. Always start with "low" and increase if needed for better accuracy.

## Setup

First, install the required packages:

In [ ]:
%pip install -qU langchain-aws

## Basic Reasoning Example

When using reasoning, the inference call may take some time depending on the complexity and reasoning effort level. We configure extended timeouts to accommodate this.

In [ ]:
from botocore.config import Config
from langchain_aws import ChatBedrockConverse

# Configure extended timeouts for reasoning
config = Config(
    connect_timeout=3600,  # 60 minutes
    read_timeout=3600,     # 60 minutes
    retries={'max_attempts': 1}
)

# Initialize model with reasoning configuration
model = ChatBedrockConverse(
    model="amazon.nova-2-lite-v1:0",
    region_name="us-east-1",
    temperature=0.7,
    max_tokens=10000,  # Set large max tokens to accommodate reasoning
    config=config,
    additional_model_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "low"
        }
    }
)

In [ ]:
# Invoke the model with a reasoning task
messages = [
    ("system", "You are a highly capable personal assistant"),
    ("human", "Provide a meal plan for a gluten free family of 4.")
]

response = model.invoke(messages)
print(response.content)

## Accessing Reasoning Content

The reasoning process is included in the response content blocks. You can access both the reasoning and the final text:

In [ ]:
# Extract reasoning content
for block in response.content:
    if isinstance(block, dict):
        if 'type' in block and block['type'] == 'reasoning_content':
            print("\n=== REASONING ===")
            print(block['reasoning_content']['text'])
        elif 'type' in block and block['type'] == 'text':
            print("\n=== RESPONSE ===")
            print(block['text'])

## Streaming with Reasoning

You can also stream responses to see the reasoning and text as they're generated:

In [ ]:
print("Streaming response:\n")

for chunk in model.stream(messages):
    if chunk.content:
        for block in chunk.content:
            if isinstance(block, dict):
                if 'type' in block and block['type'] == 'reasoning_content':
                    print(block['reasoning_content'].get('text', ''), end='', flush=True)
                elif 'type' in block and block['type'] == 'text':
                    print(block.get('text', ''), end='', flush=True)

## Tool Calling with Reasoning

Reasoning works seamlessly with tool calling. The model will reason about which tools to use:

In [ ]:
from pydantic import BaseModel, Field

class Calculator(BaseModel):
    """A calculator tool that can execute a math equation"""
    equation: str = Field(description="The full equation to evaluate")

# Bind tools to the model
model_with_tools = model.bind_tools([Calculator])

messages = [
    ("system", "For math equations you must always use the calculator tool and not your parametric knowledge"),
    ("human", "What is 2+2")
]

response = model_with_tools.invoke(messages)
print("Tool calls:", response.tool_calls)
print("\nFull response:")
print(response.content)

## Multimodal with Reasoning

Nova models support multimodal inputs including images with reasoning:

In [ ]:
import base64
from langchain_core.messages import HumanMessage

# Load and encode image
with open("path/to/your/image.jpeg", "rb") as f:
    image_data = base64.b64encode(f.read()).decode("utf-8")

# Create multimodal message
message = HumanMessage(
    content=[
        {"type": "image", "source": {"type": "base64", "media_type": "image/jpeg", "data": image_data}},
        {"type": "text", "text": "Describe the following image"}
    ]
)

response = model.invoke([message])
print(response.content)

## Different Reasoning Effort Levels

You can adjust the reasoning effort based on task complexity:

In [ ]:
# Low reasoning effort (faster, less deep thinking)
model_low = ChatBedrockConverse(
    model="amazon.nova-2-lite-v1:0",
    max_tokens=10000,
    config=config,
    additional_model_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "low"
        }
    }
)

# Medium reasoning effort (balanced)
model_medium = ChatBedrockConverse(
    model="amazon.nova-2-lite-v1:0",
    max_tokens=10000,
    config=config,
    additional_model_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "medium"
        }
    }
)

# High reasoning effort (slower, deeper thinking)
model_high = ChatBedrockConverse(
    model="amazon.nova-2-lite-v1:0",
    max_tokens=10000,
    config=config,
    additional_model_request_fields={
        "reasoningConfig": {
            "type": "enabled",
            "maxReasoningEffort": "high"
        }
    }
)

print("Try different reasoning levels based on your task complexity!")